In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import warnings, json, astropy, os
import astropy.io.fits as fits
from astropy.io.fits import getdata
import astropy.units as u
from astropy.coordinates import SkyCoord
warnings.filterwarnings("ignore")
from qa_gawa import plot_pure, plot_comp


# plt.rcParams["figure.figsize"] = (10,8)
# plt.style.use(['science' , 'notebook', 'grid'])

## Introduction

In this jupyter notebook we are working with simulated cluster detections.
There is a table of simulated clusters that were entered into a stellar field, and another table
of detections, where a positional match has already been made between the simulated and detected clusters.

In this round, 66 globular clusters with exponential density profile were simulated,
where each cluster is located at the center of a HealPix pixel with nside=64,
with the distance between one cluster and another ~1 degree and a half-light radius of ~1 arcmin.

Below is information about detections only.

In [ ]:
# Main settings:
confg = "qa_gawa.json"

# read config file
with open(confg) as fstream:
    param = json.load(fstream)

globals().update(param)

# Folder of QA results
os.system("mkdir -p " + input_detection_path + "/qa")


## Matching detections and simulations
### Reading data
Reading data from detections and simulations:

In [ ]:
det_file = input_detection_path + '/clusters.fits'
data_det = getdata(det_file)
ra_det = data_det["ra"]
dec_det = data_det["dec"]

slices_file = input_detection_path + '/dslices.fits'
data_sl = getdata(slices_file)
d_slices_pc = data_sl["dist_pc"]
mM_slices = 5 * np.log10(d_slices_pc) - 5.
print(mM_slices)
bin_size_mM = mM_slices[1] - mM_slices[0]
bins_mM = np.linspace(mM_slices[0] - bin_size_mM / 2, mM_slices[-1] + bin_size_mM / 2, len(mM_slices) + 1, endpoint=True)
print(bins_mM)

f_sim = open(input_simulation_path + '/star_clusters_simulated.dat', 'r')
data_sim = f_sim.readlines()[1:]

file_sim = input_simulation_path + '/star_clusters_simulated.dat'
ra_sim, dec_sim = np.loadtxt(file_sim, usecols=(9, 10), unpack=True)

### Matching with astropy search around sky function

In [ ]:
C_sim = SkyCoord(ra=ra_sim*u.degree, dec=dec_sim*u.degree)
C_det = SkyCoord(ra=ra_det*u.degree, dec=dec_det*u.degree)

idx_sim, idx_det, d2d, d3d = C_det.search_around_sky(C_sim, 1*u.arcmin)

idx_det_outliers = [i for i in range(len(data_det)) if i not in idx_det]

file_match = open(match_file, 'w')
print('#0-peak_id 1-ra 2-dec 3-iobj 4-jobj 5-dist_init_kpc 6-dist_err_kpc 7-dist_min_kpc 8-dist_max_kpc 9-coverfrac 10-coverfrac_bkg 11-wradius_arcmin 12-snr 13-Naper 14-Naper_tot 15-NWaper_tot 16-Naper_bkg 17-icyl 18-tile 19-slice 20-id_in_tile 21-id 22-HPX64 23-N 24-MV 25-SNR 26-N_f 27-MV_f 28-SNR_f 29-L 30-B 31-ra 32-dec 33-r_exp 34-ell 35-pa 36-mass 37-dist', file=file_match)

for i,j in zip(idx_sim, idx_det):
    print(*data_det[:][j], data_sim[i], sep=' ', file=file_match, end='')

for i in (idx_det_outliers):
    print(*data_det[i], ' -99.999 ' * len(data_sim[1].split()), sep=' ', file=file_match, end='\n')

file_match.close()

In [ ]:
ra_det, dec_det, HPX64, SNR_det, SNR_sim = np.loadtxt(match_file,
                                             usecols=(1, 2, 22, 12, 28), unpack=True) 

Below, a conditional is created where the clusters matched the simulated (confirmed)
and where clusters are just candidates.

In [ ]:
real_det = (SNR_sim > 0.)
false_positive = (SNR_sim <= 0.)

Below, a conditional is created where the clusters matched the simulated (confirmed)
and where clusters are just candidates.

In [ ]:
cm = plt.cm.get_cmap('copper_r')

fig = plt.figure(figsize=(10, 6))
sc = plt.scatter(ra_det[real_det], dec_det[real_det], c=SNR_det[real_det],
                 vmin=0, vmax=np.max(SNR_det),
                 s=100., cmap=cm, alpha=0.75, label='Simulated')
plt.scatter(ra_det[false_positive], dec_det[false_positive], c='k',
            s=200.0, marker='x', label='Not matched')
plt.colorbar(sc,label = 'SNR detection')
plt.xlim(np.max(ra_det)+0.5, np.min(ra_det)-0.5)
plt.ylim(np.min(dec_det)-0.5, np.max(dec_det)+0.5)
plt.xlabel('RA')
plt.ylabel('DEC')
plt.title('Spatial distribution of clusters detected by Gawa wrt SNR')
plt.legend()
plt.show()

We can notice that all detected and true clusters (True Positives) have high signal-to-noise ratio (SNR),
while those with low SNR are false positives (FP), represented by small blue circles.

It is important to highlight in this case that all simulated clusters were detected. There may be cases where
not all simulated clusters are detected. In this case, the code should read the clusters
simulated again to see how complete the detection is.

## Purity of detection modulus distance

Below we will calculate the detection purity given the detected distance.

In [ ]:
def plot_pure(arg_all, arg_conf, label, title, bins=20):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6))
    over = (max(np.max(arg_all), np.max(arg_conf)) - min(np.min(arg_all), np.min(arg_conf))) * 0.1
    min_ = min(np.min(arg_all), np.min(arg_conf)) - over
    max_ = max(np.max(arg_all), np.max(arg_conf)) + over
    try:
        A = ax1.hist(arg_all, bins=bins, range=(bins[0], bins[-1]), histtype='step', lw=2, label='All detections')
        B = ax1.hist(arg_conf, bins=bins, range=(bins[0], bins[-1]), histtype='stepfilled', lw=2, label='True clusters')
        pureness = B[0] / A[0]
        ax1.set_xlabel(label)
        ax1.set_ylabel( 'Number of clusters detected')
        ax1.set_xlim([min_, max_])
        ax1.legend(loc=2)
    
        ax2.step(bins, np.append(pureness[0], pureness), 'r', lw=2, label='Data')
        # ax2.step(A[1][0:-1],pureness, label='Data', color='k')
        ax2.set_xlabel(label)
        ax2.set_ylabel('Purity')
        ax2.set_ylim([0,1.2])
        ax2.set_xlim([min_, max_])
        ax2.legend()
        fig.suptitle(title)
        plt.show()
    except:
        A = ax1.hist(arg_all, bins=bins, range=(min_, max_), histtype='step', lw=2, label='All detections')
        B = ax1.hist(arg_conf, bins=bins, range=(min_, max_), histtype='stepfilled', lw=2, label='True clusters')
        pureness = B[0] / A[0]
        ax1.set_xlabel(label)
        ax1.set_ylabel( 'Number of clusters detected')
        ax1.set_xlim([min_, max_])
        ax1.legend(loc=2)

        ax2.step(A[1][0:-1], pureness, 'r', lw=2, label='Data')
        ax2.set_xlabel(label)
        ax2.set_ylabel('Purity')
        ax2.set_ylim([0,1.2])
        ax2.set_xlim([min_, max_])
        ax2.legend()
        fig.suptitle(title)
        plt.show()

In [ ]:
# 0-peak_id 1-ra 2-dec 3-iobj 4-jobj 5-dist_init_kpc 6-dist_err_kpc 7-dist_min_kpc 8-dist_max_kpc 9-coverfrac
# 10-coverfrac_bkg 11-wradius_arcmin 12-snr 13-Naper 14-Naper_tot 15-NWaper_tot 16-Naper_bkg 17-icyl 18-tile 19-slice
# 20-id_in_tile 21-id 22-HPX64 23-N 24-MV 25-SNR 26-N_f 27-MV_f 28-SNR_f 29-L
# 30-B 31-ra 32-dec 33-r_exp 34-ell 35-pa 36-mass 37-dist'
dist_kpc_det, disterr_kpc_det = np.loadtxt(match_file, usecols=(5, 6), unpack=True)

m_M_det = 5 * np.log10(dist_kpc_det) + 10.

plot_pure(m_M_det, m_M_det[real_det], 'Detection distance module', 'Detection distance module(pureness)', bins_mM)

In [ ]:
plot_pure(SNR_det, SNR_det[real_det], 'Signal-to-noise ratio (detection)', 'Signal-to-noise ratio(pureness)')

In [ ]:
ipix, Nstar, M_V, SNR, L, B, RA_pix, DEC_pix, r_exp_pc, ell, pa, mass, dist = np.loadtxt(input_simulation_path + '/star_clusters_simulated.dat',
                                                                                         usecols=(0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15),
                                                                                         unpack=True)

plot_comp(M_V, idx_sim, 'M_V', 'Absolute Magnitude in V band')

In [ ]:
plot_comp(dist, idx_sim, 'r (kpc)', 'Distance')

In [ ]:
plot_comp(SNR, idx_sim, 'SNR', 'Signal to Noise Ratio')

In [ ]:
mM_sim = 5 * np.log10(dist) - 5.

plot_comp(mM_sim, idx_sim, 'm-M', 'Distance modulus')